# 대화 요약을 위한 인스트럭터 모델(Instructor-Model) 미세 조정하기

## 준비사항
아래 실습은 AWS ml.m5.2xlarge 인스턴스에서 수행했습니다.

## 커널 및 필요한 종속성 설정하기

In [2]:
%pip install --disable-pip-version-check \
    torch==2.0.1 \
    transformers==4.34.1 \
    datasets==2.12.0 \
    accelerate==0.23.0 \
    evaluate==0.4.0 \
    py7zr==0.20.4 \
    sentencepiece==0.1.99 \
    rouge_score==0.1.2 \
    loralib==0.1.1 \
    peft==0.4.0 \
    trl==0.7.2

Note: you may need to restart the kernel to use updated packages.


In [3]:
model_checkpoint='google/flan-t5-base'

In [4]:
# 이 디렉터리는 이전 노트북에서 생성됐습니다.
local_data_processed_path = './data-summarization-processed/'

# 패키지 적재

In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer, GenerationConfig
from datasets import load_dataset
import datasets
import torch
import time
import evaluate
import numpy as np
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# 허깅 페이스 모델 적재

허깅 페이스에서 사전 학습된 Flan-T5 모델을 직접 불러올 수 있습니다. 이번 실습에서는 Flan 기본 버전을 사용합니다. 이 버전은 약 2억 4,700만 개의 파라미터를 가지고 있어 다른 대형 언어 모델(LLM)에 비해 작습니다. 더 높은 품질의 결과를 원한다면 이 모델의 더 큰 버전들을 고려하는 것이 좋습니다.

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [7]:
params = sum(p.numel() for p in model.parameters())
print(f'Total Number of Model Parameters: {params}')

Total Number of Model Parameters: 247577856


# 처리된 데이터 적재

# 데이터 세트 적재

미리 처리된 `DialogSum` 데이터 세트는 로컬 디렉터리에서 직접 불러올 수 있습니다. 이 데이터 세트에는 약 15,000개의 대화 예시와 그에 따른 사람이 작성한 요약이 포함되어 있습니다.

In [8]:
tokenized_dataset = load_dataset(
    local_data_processed_path,
    data_files={'train': 'train/*.parquet', 'test': 'test/*.parquet', 'validation': 'validation/*.parquet'}
).with_format("torch")
tokenized_dataset

Found cached dataset parquet (/root/.cache/huggingface/datasets/parquet/data-summarization-processed-38e385bef3c7b73c/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 13014
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 723
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 723
    })
})

# 미세 조정 전 제로샷 프롬프트로 모델 테스트하기

아래 예시에서는 데이터 세트에 제공된 기본 요약과 비교해 모델의 요약 기능이 부족함을 보여줍니다. 모델은 기본 요약과 비교했을 때 대화를 요약하는 데 어려움을 겪지만, 일부 중요한 정보를 텍스트에서 추출합니다. 이는 모델이 주어진 작업에 맞게 미세 조정될 수 있음을 시사합니다.

In [9]:
idx = 2
diag = tokenizer.decode(tokenized_dataset['test'][idx]['input_ids'], skip_special_tokens=True)
model_input = tokenizer(diag, return_tensors="pt").input_ids
summary = tokenizer.decode(tokenized_dataset['test'][idx]['labels'], skip_special_tokens=True)

original_outputs = model.to('cpu').generate(model_input, GenerationConfig(max_new_tokens=200))
original_text_output = tokenizer.decode(original_outputs[0], skip_special_tokens=True)

diag_print = diag.replace(' #',' \n#')
print(f"Prompt:\n--------------------------\n{diag_print}\n--------------------------")
print(f'\nOriginal Model Response: {original_text_output}')
print(f'Baseline Summary : {summary}')

Prompt:
--------------------------
Summarize the following conversation. 
#Person1#: Hello. My name is John Sandals, and I've got a reservation. 
#Person2#: May I see some identification, sir, please? 
#Person1#: Sure. Here you are. 
#Person2#: Thank you so much. Have you got a credit card, Mr. Sandals? 
#Person1#: I sure do. How about American Express? 
#Person2#: Unfortunately, at the present time we take only MasterCard or VISA. 
#Person1#: No American Express? Okay, here's my VISA. 
#Person2#: Thank you, sir. You'll be in room 507, nonsmoking, with a queen-size bed. Do you approve, sir? 
#Person1#: Yeah, that'll be fine. 
#Person2#: That's great. This is your key, sir. If you need anything at all, anytime, just dial zero. Summary: 
--------------------------

Original Model Response: John Sandals has a reservation for a room at the Venetian Hotel in Las Vegas.
Baseline Summary : John Sandals has got a reservation. #Person1# asks for his identification and credit card and helps his 

# 인스트럭터 모델 미세 조정하기

데이터 세트를 전처리했으니 허깅 페이스 내장 `Trainer` 클래스로 주어진 작업에 맞게 모델을 미세 조정할 수 있습니다. 전체 모델을 학습하는 데는 GPU에서 몇 시간이 걸리므로, 시간을 절약하려면 다운샘플링 없이 10번의 에포크 동안 학습된 모델 체크포인트를 참고하세요. 모델을 직접 완전히 학습하고 싶다면 코드 변경 방법에 대한 인라인 주석을 참고하면 됩니다. GPU로 학습하려면 제공된 체크포인트는 `ml.g5.xlarge` 인스턴스를 사용했습니다.

In [10]:
# 실습 시간 절약을 위해, 데이터 세트 부분 샘플링
# 모델을 완전히 학습시키고 싶다면, 부분 샘플링을 수정해 더 큰 데이터 세트를 생성하는게 좋습니다.
sample_tokenized_dataset = tokenized_dataset.filter(lambda example, indice: indice % 100 == 0, with_indices=True)

output_dir = f'./diag-summary-training-{str(int(time.time()))}'
training_args = TrainingArguments(
    output_dir=output_dir,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-5,
    num_train_epochs=1,
    # num_train_epochs=10, # 실험할 시간이 더 많을 때는 더 많은 에포크 수를 사용하세요.
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=sample_tokenized_dataset['train'],
    eval_dataset=sample_tokenized_dataset['validation']
)

Filter:   0%|          | 0/13014 [00:00<?, ? examples/s]

Filter:   0%|          | 0/723 [00:00<?, ? examples/s]

Filter:   0%|          | 0/723 [00:00<?, ? examples/s]

In [11]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,35.780022


TrainOutput(global_step=33, training_loss=38.12218498461174, metrics={'train_runtime': 418.7129, 'train_samples_per_second': 0.313, 'train_steps_per_second': 0.079, 'total_flos': 89703213170688.0, 'train_loss': 38.12218498461174, 'epoch': 1.0})

# 학습된 모델과 원본 모델 적재

모델 학습이 완료되면 허깅 페이스 원본 모델과 미세 조정 모델을 불러와 질적·양적 비교를 합니다.

In [12]:
!aws s3 cp --recursive s3://dsoaws/models/flan-dialogue-summary-checkpoint/ ./flan-dialogue-summary-checkpoint/

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


download: s3://dsoaws/models/flan-dialogue-summary-checkpoint/generation_config.json to flan-dialogue-summary-checkpoint/generation_config.json
download: s3://dsoaws/models/flan-dialogue-summary-checkpoint/trainer_state.json to flan-dialogue-summary-checkpoint/trainer_state.json
download: s3://dsoaws/models/flan-dialogue-summary-checkpoint/rng_state.pth to flan-dialogue-summary-checkpoint/rng_state.pth
download: s3://dsoaws/models/flan-dialogue-summary-checkpoint/scheduler.pt to flan-dialogue-summary-checkpoint/scheduler.pt
download: s3://dsoaws/models/flan-dialogue-summary-checkpoint/config.json to flan-dialogue-summary-checkpoint/config.json
download: s3://dsoaws/models/flan-dialogue-summary-checkpoint/training_args.bin to flan-dialogue-summary-checkpoint/training_args.bin
download: s3://dsoaws/models/flan-dialogue-summary-checkpoint/pytorch_model.bin to flan-dialogue-summary-checkpoint/pytorch_model.bin
download: s3://dsoaws/models/flan-dialogue-summary-checkpoint/optimizer.pt to fl

In [13]:
# 직접 모델을 학습하고 우리의 모델과 비교해 보고 싶다면, 아래 코드를 변경하세요.
# 여러분의 체크포인트 디렉터리로 변경하세요.

supervised_fine_tuned_model_path = "./flan-dialogue-summary-checkpoint"
# supervised_fine_tuned_model_path = f"./{output_dir}/<put-your-checkpoint-dir-here>"

tuned_model = AutoModelForSeq2SeqLM.from_pretrained(supervised_fine_tuned_model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [14]:
%store supervised_fine_tuned_model_path

Stored 'supervised_fine_tuned_model_path' (str)


# 미세 조정 후 제로샷 추론으로 정성적 결과 평가하기

많은 생성형 AI 애플리케이션처럼 "내 모델이 의도한 대로 작동하는가?"라는 질문을 스스로 던지는 정성적 접근법이 일반적으로 좋은 출발점입니다. 아래 예시(이 노트북을 시작할 때 사용한 것과 같은 예시)에서 미세 조정 모델이 원래 모델이 요구 사항을 이해하지 못했던 것과 달리 대화의 합리적인 요약을 만들 수 있음을 알 수 있습니다.

In [15]:
idx = 2
diag = tokenizer.decode(tokenized_dataset['test'][idx]['input_ids'], skip_special_tokens=True)
model_input = tokenizer(diag, return_tensors="pt").input_ids
summary = tokenizer.decode(tokenized_dataset['test'][idx]['labels'], skip_special_tokens=True)

original_outputs = model.to('cpu').generate(
    model_input,
    GenerationConfig(max_new_tokens=200, num_beams=1),
)
outputs = tuned_model.to('cpu').generate(
    model_input,
    GenerationConfig(max_new_tokens=200, num_beams=1,),
)
text_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

diag_print = diag.replace(' #',' \n#')
print(f"Prompt:\n--------------------------\n{diag_print}\n--------------------------")
print(f'Flan-T5 response: {original_text_output}')
print(f'Our instruct-tuned response (on top of Flan-T5): {text_output}')
print(f'Baseline summary from original dataset: {summary}')

Prompt:
--------------------------
Summarize the following conversation. 
#Person1#: Hello. My name is John Sandals, and I've got a reservation. 
#Person2#: May I see some identification, sir, please? 
#Person1#: Sure. Here you are. 
#Person2#: Thank you so much. Have you got a credit card, Mr. Sandals? 
#Person1#: I sure do. How about American Express? 
#Person2#: Unfortunately, at the present time we take only MasterCard or VISA. 
#Person1#: No American Express? Okay, here's my VISA. 
#Person2#: Thank you, sir. You'll be in room 507, nonsmoking, with a queen-size bed. Do you approve, sir? 
#Person1#: Yeah, that'll be fine. 
#Person2#: That's great. This is your key, sir. If you need anything at all, anytime, just dial zero. Summary: 
--------------------------
Flan-T5 response: John Sandals has a reservation for a room at the Venetian Hotel in Las Vegas.
Our instruct-tuned response (on top of Flan-T5): John Sandals has a reservation and checks in with his VISA. #Person2# helps him to

# ROUGE 지표를 사용한 정량적 결과 평가

[ROUGE 지표](https://en.wikipedia.org/wiki/ROUGE_(metric))는 모델이 생성한 요약의 유효성을 정량화하는 데 도움이 됩니다. 이 지표는 모델이 생성한 요약을 사람이 작성한 ‘기준’ 요약과 비교합니다. 완벽하진 않지만, 미세 조정을 통해 전반적인 요약 효과성이 향상된 것을 보여줍니다.

In [16]:
rouge = evaluate.load('rouge')

## 요약의 하위 섹션 평가

In [17]:
# 시간을 절약하기 위해 각 모델로 몇 개의 요약만 생성합니다.
# 실습실 외부에서는 생성할 검증 요약의 수를 늘려보는 것이 좋습니다.
dialogues = tokenized_dataset['test'][0:10]['input_ids']
baseline_summaries = tokenized_dataset['test'][0:10]['labels']

# 원본 요약 디코딩
human_baseline_summaries = []
for base_summary in baseline_summaries:
    human_baseline_summaries.append(tokenizer.decode(base_summary, skip_special_tokens=True))

# 요약 생성
original_outputs = model.generate(dialogues, GenerationConfig(max_new_tokens=200))
tuned_outputs = tuned_model.generate(dialogues, GenerationConfig(max_new_tokens=200))

In [18]:
# 요약을 리스트에 저장
original_model_summaries = []
tuned_model_summaries = []

# 모든 요약 디코딩
for original_summary, tuned_summary in zip(original_outputs, tuned_outputs):
    original_model_summaries.append(tokenizer.decode(original_summary, skip_special_tokens=True))
    tuned_model_summaries.append(tokenizer.decode(tuned_summary, skip_special_tokens=True))

In [19]:
original_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries,
    use_aggregator=True,
    use_stemmer=True,
)

In [20]:
tuned_results = rouge.compute(
    predictions=tuned_model_summaries,
    references=human_baseline_summaries,
    use_aggregator=True,
    use_stemmer=True,
)

In [21]:
original_results

{'rouge1': 0.23899693678641046,
 'rouge2': 0.08932806324110672,
 'rougeL': 0.21776705653021444,
 'rougeLsum': 0.21325132275132275}

In [22]:
tuned_results

{'rouge1': 0.5243400833160587,
 'rouge2': 0.240785255433749,
 'rougeL': 0.3960164115550142,
 'rougeLsum': 0.396040090323604}

## 전체 데이터 세트 평가

"diag-summary-training-results.csv" 라는 파일에는 더 큰 규모의 데이터에 대해 평가할 수 있는 모든 모델 결과가 미리 채워져 있습니다. 결과는 모든 ROUGE 지표에서 상당한 개선을 보여줍니다!

In [23]:
import pandas as pd
results = pd.read_csv("diag-summary-training-results.csv")
original_model_summaries = results['original_model_summaries'].values
tuned_model_summaries = results['tuned_model_summaries'].values
human_baseline_summaries = results['human_baseline_summaries'].values

In [24]:
original_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

In [25]:
tuned_results = rouge.compute(
    predictions=tuned_model_summaries,
    references=human_baseline_summaries[0:len(tuned_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

In [26]:
original_results

{'rouge1': 0.2334158581572823,
 'rouge2': 0.07603964187010573,
 'rougeL': 0.20145520923859048,
 'rougeLsum': 0.20145899339006135}

In [27]:
tuned_results

{'rouge1': 0.42161291557556113,
 'rouge2': 0.18035380596301792,
 'rougeL': 0.3384439349963909,
 'rougeLsum': 0.33835653595561666}

In [28]:
improvement = (np.array(list(tuned_results.values())) - np.array(list(original_results.values())))
for key, value in zip(tuned_results.keys(), improvement):
    print(f'{key} absolute percentage improvement after instruct fine-tuning: {value*100:.2f}%')

rouge1 absolute percentage improvement after instruct fine-tuning: 18.82%
rouge2 absolute percentage improvement after instruct fine-tuning: 10.43%
rougeL absolute percentage improvement after instruct fine-tuning: 13.70%
rougeLsum absolute percentage improvement after instruct fine-tuning: 13.69%
